# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Jiahao Xu</p>
### <p style="text-align: right;"> &#9989; boyang0303</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [10]:
#Write your code here
import pandas as pd

data = pd.read_csv("aussie_weather_data.csv")
data.head()


,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [15]:
#Write your code here
import pandas as pd

cities = data['Location'].unique()
print(cities)


['NorfolkIsland' 'SydneyAirport' 'WaggaWagga' 'MelbourneAirport' 'Mildura'
 'Watsonia' 'Brisbane' 'Cairns' 'Townsville' 'MountGambier' 'Nuriootpa'
 'PerthAirport' 'Perth' 'AliceSprings' 'Darwin']


#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [17]:
#Write your code here
selected_city = "WaggaWagga"
city_data = data[data['Location'] == selected_city]
city_data.head()



,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
5334,2009,1,1,WaggaWagga,9.9,28.4,0.0,9.8,13.5,157.5,...,37.0,24.0,1005.2,1003.5,1.0,1.0,20.8,27.0,0,0
5335,2009,1,2,WaggaWagga,8.9,25.3,0.0,14.8,13.7,202.5,...,37.0,10.0,1013.9,1012.9,1.0,0.0,14.7,23.2,0,0
5336,2009,1,3,WaggaWagga,11.0,30.7,0.0,9.8,13.7,67.5,...,43.0,18.0,1017.8,1014.3,0.0,1.0,18.9,28.0,0,0
5337,2009,1,4,WaggaWagga,14.7,35.2,0.0,9.4,12.1,90.0,...,46.0,19.0,1014.8,1010.6,1.0,4.0,24.7,33.7,0,0
5338,2009,1,5,WaggaWagga,11.8,36.5,0.0,11.6,13.4,180.0,...,23.0,5.0,1013.1,1009.8,1.0,1.0,23.6,34.6,0,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [38]:
#Write your code here
from sklearn.model_selection import train_test_split

X = city_data[['Rainfall', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm']]
y = city_data['RainTomorrow']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=72)


print("Training dataset (X_train):", X_train.shape)
print("Testing dataset (X_test):", X_test.shape)


Training dataset (X_train): (1691, 6)
Testing dataset (X_test): (725, 6)


## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

For binary classification problems, logistic regression is more appropriate than linear regression.

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [39]:
#Write your code here
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
feature_names = X.columns
coefficients = model.coef_[0]

for feature, coef in zip(feature_names, coefficients):
    print(f"{feature}: {coef}")

Rainfall: 0.043602335004060085
Humidity3pm: 0.045902696855785005
Pressure9am: 0.3506789664279133
Pressure3pm: -0.3576696644413231
Cloud9am: 0.11312725548435458
Cloud3pm: 0.3331465044773388


 Pressure3pm, Pressure9am, and Cloud3pm seem to be the most important features in this logistic regression model for predicting rain tomorrow.

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [40]:
#Write your code here


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.8896551724137931
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.97      0.93       588
           1       0.81      0.54      0.65       137

    accuracy                           0.89       725
   macro avg       0.86      0.76      0.79       725
weighted avg       0.88      0.89      0.88       725

Confusion Matrix:
[[571  17]
 [ 63  74]]


#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

For Class 0 (no rain tomorrow), the model has high precision (0.90) and higher recall (0.95), which means the model is good at predicting the absence of rain and does not misclassify many non-rainy days as rainy.

For Class 1 (rain tomorrow), the model has moderate precision (0.70) but lower recall (0.51). This means that when the model predicts rain tomorrow, it is correct 70% of the time. However, it only correctly identifies 51% of the actual rainy days. In other words, the model tends to miss many rainy days and classify them as non-rainy days.

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [41]:
#Write your code for calculating the frequency of rain here.


total_days = len(city_data)
rainy_days = len(city_data[city_data['RainTomorrow'] == 1])
no_rain_days = len(city_data[city_data['RainTomorrow'] == 0])

rain_frequency = rainy_days / total_days

no_rain_frequency = no_rain_days / total_days

print(f"Rain frequency: {rain_frequency:.2f}")
print(f"No rain frequency: {no_rain_frequency:.2f}")

Rain frequency: 0.18
No rain frequency: 0.82


If I were to guess that it wasn't going to rain every day, I will be 82% right

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [42]:
#Write your code here
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0],
    'gamma': [0.01, 0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf']
}


svm_classifier = SVC()


grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, n_jobs=-1)


grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_

print("Best parameters found:")
for param, value in best_params.items():
    print(f"{param}: {value}")

Best parameters found:
C: 10.0
gamma: 0.01
kernel: linear


Best parameters found:
C: 0.01
gamma: 0.01
kernel: linear

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [43]:
#Write your code here
best_svm_classifier = SVC(**best_params)

best_svm_classifier.fit(X_train, y_train)

y_pred = best_svm_classifier.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)

class_report = classification_report(y_test, y_pred, output_dict=True)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print(f"Overall Accuracy: {accuracy:.2f}")

Confusion Matrix:
[[569  19]
 [ 54  83]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       588
           1       0.81      0.61      0.69       137

    accuracy                           0.90       725
   macro avg       0.86      0.79      0.82       725
weighted avg       0.89      0.90      0.89       725

Overall Accuracy: 0.90


## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


The regression and SVM models perform better than the baseline model, as their accuracy values are higher. The performance of the SVM model is slightly better than that of the regression model, with an accuracy of 0.88 compared to 0.87. This indicates that the SVM model is better at predicting whether it will rain tomorrow based on the given features.

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


Regeression Classification Report: Seed 42

              precision    recall  f1-score   support

           0       0.90      0.95      0.92       594
           1       0.70      0.51      0.59       131
    accuracy                           0.87       725

SVM Classification Report: Seed 42

              precision    recall  f1-score   support

           0       0.90      0.97      0.93       594
           1       0.77      0.50      0.60       131

    accuracy                           0.88       725
    
Regeression Classification Report: Seed 88

              precision    recall  f1-score   support

           0       0.88      0.96      0.92       587
           1       0.74      0.45      0.56       138

    accuracy                           0.86       725
    

SVM Classification Report: Seed 88

              precision    recall  f1-score   support

           0       0.90      0.97      0.93       587
           1       0.79      0.53      0.63       138

    accuracy                           0.88       725
    
Regeression Classification Report: Seed 72

              precision    recall  f1-score   support

           0       0.90      0.97      0.93       588
           1       0.81      0.54      0.65       137

    accuracy                           0.89       725
    
    
SVM Classification Report: Seed 72

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       588
           1       0.81      0.61      0.69       137

    accuracy                           0.90       725

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

From the provided information, we can observe that there is some variation in the precision, recall, and accuracy values across different random seeds for both the regression and SVM models. The variation could be due to the randomness introduced in the training and test data split or the optimization of hyperparameters during the training process.

After going through this exercise, the variation in the model performance does not change our response to part 5.1. We still conclude that both the regression and SVM models perform better than the baseline model, and the SVM model performs slightly better than the regression model in predicting whether it will rain tomorrow based on the given features.


#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

The SVM model performs slightly better than the regression model in predicting whether it will rain tomorrow based on the given features. also the SVM model had a higher precision and recall value for the minority class which indicating that it was better at correctly identifying the instances where it will rain tomorrow.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University